In [5]:
import pandas as pd
# vcf = pd.read_pickle("all_vcfs.pkl")
c = pd.read_csv("/Users/ishaponugoti/Desktop/DL AD/ADDetection/preprocess_clinical/clinical.csv").drop("Unnamed: 0", axis=1).rename(columns={"PTID":"subject"})
img = pd.read_pickle("/Users/ishaponugoti/Desktop/DL AD/ADDetection/preprocess_images/mri_meta.pkl")[["img_array", "subject", "label"]]

In [115]:
# vcf.head()

In [6]:
c.head()

,subject,RID,Phase,GroupN,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4,PTHOME_-4.0,PTHOME_1.0,...,PTETHCAT_2.0,PTETHCAT_3.0,PTDOBYY,PHC_MEM,PHC_EXF,PTRACCAT,PTETHCAT,PTCOGBEG,PHC_VSP,PHC_LAN
0,002_S_0295,295,ADNI1,0.0,False,True,False,True,False,True,...,True,False,1921.0,1.463,0.554,5,2.0,NaN,0.483,0.272
1,002_S_0413,413,ADNI1,0.0,False,False,True,True,False,True,...,True,False,1929.0,1.227,1.249,5,2.0,NaN,0.483,2.188
2,002_S_0559,559,ADNI1,0.0,False,True,False,True,False,True,...,True,False,1927.0,1.110,0.892,5,2.0,NaN,0.963,0.816
3,002_S_0619,619,ADNI1,2.0,False,True,False,True,False,False,...,True,False,1928.0,-1.105,-0.525,5,2.0,NaN,-0.422,-0.072
4,002_S_0685,685,ADNI1,0.0,False,False,True,True,False,True,...,True,False,1916.0,0.667,0.602,5,2.0,NaN,0.963,0.682


In [7]:
c = c.rename(columns = {"Group":"GROUP"})

In [9]:
# a = vcf.merge(c, on = ["subject", "GROUP"]).merge(img, on = "subject")
a = c

In [10]:
a["subject"].value_counts()

subject
002_S_0295    1
099_S_6038    1
099_S_6097    1
099_S_6175    1
099_S_6344    1
             ..
029_S_1215    1
029_S_1218    1
029_S_1318    1
029_S_1384    1
941_S_7106    1
Name: count, Length: 2925, dtype: int64

In [11]:
a["GroupN"].value_counts()

GroupN
0.0    1077
1.0     969
2.0     879
Name: count, dtype: int64

In [121]:
# a["GROUP"].value_counts()

In [12]:
cols = list(set(a.columns) - set(["PTID", "label", "GROUP",
                                  "RID", "ID", "Group", "Phase", "SITEID", "VISCODE", "VISCODE2", "USERDATE", "USERDATE2", "update_stamp", "DX", "Unnamed: 0"]))

In [13]:
X= a[cols]
y = a["GroupN"]

In [20]:
img = img[~img['label'].isin([3, 4, 5])]

# Now 'filtered_df' contains only the rows where label is not 3, 4, or 5.
print(img)

                                             img_array     subject label
0    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  109_S_1157     1
1    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  027_S_0835     0
2    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  023_S_0217     0
3    [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  127_S_0925     0
4    [[[0.0, 0.1871623869174006, 0.0], [0.0, 0.0531...  018_S_4399     2
..                                                 ...         ...   ...
311  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  067_S_0257     2
312  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  114_S_0378     0
313  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  136_S_0186     2
314  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  023_S_0042     0
316  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...  133_S_0727     0

[299 rows x 3 columns]


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

In [22]:
X_test[["subject"]].to_csv("overlap_test_set.csv")

In [126]:
# snp_cols = set(X_train.columns).intersection(set(vcf.columns))

In [127]:
# X_train_snp = X_train[snp_cols]
# X_test_snp = X_test[snp_cols]

In [26]:
img_cols = list(set(X_train.columns).intersection(set(img.columns)))
print(len(img.columns))
print(img.columns)
X_train_img = X_train[img_cols]
X_test_img = X_test[img_cols]

3
Index(['img_array', 'subject', 'label'], dtype='object')


In [27]:
clin_cols = list(set(X_train.columns).intersection(set(c.columns)))
print(len(c.columns))
print(len(clin_cols))
X_train_clin = X_train[clin_cols]
X_test_clin = X_test[clin_cols]

104
102


In [31]:
import numpy as np
print("Unique labels in training data:", np.unique(y_train))

Unique labels in training data: [0. 1. 2.]


In [32]:
# pd.DataFrame(X_train_snp).to_pickle("X_train_snp.pkl")
# pd.DataFrame(X_test_snp).to_pickle("X_test_snp.pkl")
pd.DataFrame(y_train).to_pickle("y_train.pkl")
pd.DataFrame(y_test).to_pickle("y_test.pkl")

pd.DataFrame(X_train_clin).to_pickle("X_train_clinical.pkl")
pd.DataFrame(X_test_clin).to_pickle("X_test_clinical.pkl")

pd.DataFrame(X_train_img).to_pickle("X_train_img.pkl")
pd.DataFrame(X_test_img).to_pickle("X_test_img.pkl")

---------------------------------------------------------------------------------------------------------